# Transfer learning : MobileNet

#Training and Predicting with MobileNetV2 for Multi-Class Image Classification

About:
This code trains a deep learning model using MobileNetV2 for classifying images into three categories. It utilizes transfer learning with a pre-trained MobileNetV2 model, applies data augmentation, and includes early stopping for efficient training. After training the model, it evaluates the performance on a test set, saves the trained model, and predicts the class of new images. The results are visualized using matplotlib to display predictions alongside the corresponding images.

1. valiadtion accuracy 87

2. validation accuracy 93

In [ ]:
# Import necessary libraries
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

# Mount Google Drive to access datasets
drive.mount('/content/drive')

# Install TensorFlow if not already installed
!pip install tensorflow

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Define image dimensions (MobileNetV2 standard input size)
img_height, img_width = 224, 224

# Data augmentation and normalization using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define dataset directories
train_dir = '/content/drive/MyDrive/Classification/classidata/train'
valid_dir = '/content/drive/MyDrive/Classification/classidata/valid'
test_dir = '/content/drive/MyDrive/Classification/classidata/test'

# Load datasets using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

# Load MobileNetV2 pre-trained model without the top layers (for transfer learning)
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,  # Exclude the top layers (fully connected)
                                               weights='imagenet')  # Load pre-trained weights from ImageNet

# Freeze the base model (to prevent re-training the base layers)
base_model.trainable = False

# Create the new model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Global pooling instead of flatten
    layers.Dense(256, activation='relu'),  # Dense layer
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(3, activation='softmax')  # Output layer for 3 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback when validation accuracy reaches above 90%
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                  patience=3,  # Number of epochs to wait after reaching 90%
                                                  min_delta=0.01,  # Minimum change to qualify as an improvement
                                                  mode='max',  # Stop when the validation accuracy reaches max value
                                                  verbose=1,
                                                  restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,  # Number of batches per epoch
    epochs=15,  # Set the number of epochs to 15
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // 32,  # Number of validation batches
    callbacks=[early_stopping]  # Include early stopping callback
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Save the model
model.save('/content/drive/MyDrive/Classification/classidata/best.h5')
print('Model saved!')

# -------------------------------------------------------------
import matplotlib.pyplot as plt
from keras.preprocessing import image
import os
import numpy as np

# Define path to new images for prediction
new_image_dir = '/content/drive/MyDrive/ooo'  # Replace with your new image directory path

# Function to predict new images and display them
def predict_new_images(image_dir):
    # List all image files in the directory
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    # Iterate over each image
    for img_file in image_files:
        img_path = os.path.join(image_dir, img_file)

        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(img_height, img_width))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array /= 255.0  # Normalize the image

        # Make prediction
        predictions = model.predict(img_array)

        # Get the predicted class (index of max probability)
        predicted_class = np.argmax(predictions, axis=1)

        # Get the class label
        class_labels = list(train_generator.class_indices.keys())  # This gives you the class names from your training data
        predicted_label = class_labels[predicted_class[0]]

        # Display the image and prediction result
        plt.imshow(img)
        plt.title(f"Prediction: {predicted_label}")
        plt.axis('off')  # Hide axes for a cleaner display
        plt.show()

# Call the function to make predictions
predict_new_images(new_image_dir)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Import necessary libraries
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

# Mount Google Drive to access datasets
drive.mount('/content/drive')

# Install TensorFlow if not already installed
!pip install tensorflow

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Define image dimensions (MobileNetV2 standard input size)
img_height, img_width = 224, 224

# Data augmentation and normalization using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define dataset directories
train_dir = '/content/drive/MyDrive/Classification/classidata/train'
valid_dir = '/content/drive/MyDrive/Classification/classidata/valid'
test_dir = '/content/drive/MyDrive/Classification/classidata/test'

# Load datasets using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)

# Load MobileNetV2 pre-trained model without the top layers (for transfer learning)
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,  # Exclude the top layers (fully connected)
                                               weights='imagenet')  # Load pre-trained weights from ImageNet

# Freeze the base model (to prevent re-training the base layers)
base_model.trainable = False

# Create the new model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Global pooling instead of flatten
    layers.Dense(256, activation='relu'),  # Dense layer
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(3, activation='softmax')  # Output layer for 3 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,  # Number of batches per epoch
    epochs=15,  # Set the number of epochs to 15
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // 32  # Number of validation batches
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Save the model
model.save('/content/drive/MyDrive/Classification/classidata/mobilenetv2_best.h5')
print('Model saved!')

# -------------------------------------------------------------


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 4782 images belonging to 3 classes.
Found 1026 images belonging to 3 classes.
Found 1026 images belonging to 3 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 2047s 13s/step - accuracy: 0.6459 - loss: 0.8548 - val_accuracy: 0.8447 - val_loss: 0.3735
Epoch 2/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8438 - loss: 0.3266 - val_accuracy: 0.5000 - val_loss: 0.5471
Epoch 3/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 363s 2s/step - accuracy: 0.8476 - loss: 0.4173 - val_accuracy: 0.8809 - val_loss: 0.3058
Epoch 4/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 729us/step - accuracy: 0.8438 - loss: 0.2611 - val_accuracy: 1.0000 - val_loss: 0.2463
Epoch 5/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 359s 2s/step - accuracy: 0.8808 - loss: 0.3183 - val_accuracy: 0.8779 - val_loss: 0.2957
Epoch 6/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/st

Test Accuracy: 93.96%
Model saved!


In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing import image
import os
import numpy as np

# Define path to new images for prediction
new_image_dir = '/content/drive/MyDrive/ooo'  # Replace with your new image directory path

# Function to predict new images and display them
def predict_new_images(image_dir):
    # List all image files in the directory
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    # Iterate over each image
    for img_file in image_files:
        img_path = os.path.join(image_dir, img_file)

        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(img_height, img_width))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array /= 255.0  # Normalize the image

        # Make prediction
        predictions = model.predict(img_array)

        # Get the predicted class (index of max probability)
        predicted_class = np.argmax(predictions, axis=1)

        # Get the class label
        class_labels = list(train_generator.class_indices.keys())  # This gives you the class names from your training data
        predicted_label = class_labels[predicted_class[0]]

        # Display the image and prediction result
        plt.imshow(img)
        plt.title(f"Prediction: {predicted_label}")
        plt.axis('off')  # Hide axes for a cleaner display
        plt.show()

# Call the function to make predictions
predict_new_images(new_image_dir)


Output hidden; open in https://colab.research.google.com to view.